# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [7]:
import openai
import json
openai.api_key = ""

In [5]:
# The goal of the Chatbot is to answer the questions about advancements in GPU technology
# used for AI application development. I have chosen Wiki page about NVIDIA Blackwell GPU for training the chatbot.
# This Wiki page is appropriate for this task due to following reasons
# 1. Blackwell GPU is the latest advance GPU announced by NVIDIA.
# 2. The announcement happened recently in NVIDIA GTC 2024. Therfore ChatGPT wouldn't have trained with this information

from dateutil.parser import parse
import pandas as pd
import requests

# In this project, I selected the Blackwell GPU Wiki page for our custom chatbot development 
# as this is the advanced GPU developed by NVIDIA so far. This dataset is particularly suitable
# because goal of the chatbot is to converse on all NVIDIA GPU architectures, including the latest one.
# GPT3.5 is used as a base mdoel for the chatbot and GPT3.5 has trained on data before year 2022.
# So, training on latest NVIDIA GPUs will help the Chatbot to communicate on latest NVIDIA GPUs too.

resp = requests.get("https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exlimit=1&titles=Blackwell_(microarchitecture)&explaintext=1&formatversion=2&format=json")

# Load page text into a dataframe
df = pd.DataFrame()
df["text"] = resp.json()["query"]["pages"][0]["extract"].split("\n")
#print(df["text"])
# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]
df = df[~(df["text"].str.contains("\t\t") | df["text"].str.contains("\t\t\t"))]
# In some cases dates are used as headings instead of being part of the
# text sample; adjust so dated text samples start with dates
prefix = ""
for (i, row) in df.iterrows():
    # If the row already has " - ", it already has the needed date prefix
    if " – " not in row["text"]:
        try:
            # If the row's text is a date, set it as the new prefix
            parse(row["text"])
            prefix = row["text"]
        except:
            # If the row's text isn't a date, add the prefix
            row["text"] = prefix + " – " + row["text"]
df = df[df["text"].str.contains(" – ")]
df


0     Blackwell is a graphics processing unit (GPU) ...
1     Named after statistician and mathematician Dav...
2                                                      
3                                                      
4                                         == History ==
5                                                      
6     In March 2022, Nvidia announced Hopper archite...
7     The Blackwell architecture is named after Amer...
8     In Nvidia's October 2023 Investor Presentation...
9     At the Graphics Technology Conference (GTC) on...
10                                                     
11                                                     
12                                   == Architecture ==
13    Blackwell is an architecture designed for both...
14                                                     
15                                                     
16                                 === Process node ===
17    Blackwell is fabricated on the custom 4NP 

,text
0,– Blackwell is a graphics processing unit (GP...
1,– Named after statistician and mathematician ...
6,"– In March 2022, Nvidia announced Hopper arch..."
7,– The Blackwell architecture is named after A...
8,– In Nvidia's October 2023 Investor Presentat...
9,– At the Graphics Technology Conference (GTC)...
13,– Blackwell is an architecture designed for b...
17,– Blackwell is fabricated on the custom 4NP n...
18,– The GB100 die is at the reticle limit of se...
25,– CUDA Compute Capability 10.0 is added with ...


In [8]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 10
embeddings = []
#for i in range(0, len(df), batch_size):
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    print("Before OpenAI Request : ",i)
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    #print("OpenAI Respose : ",response.status_code)
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df
df.to_csv("embeddings.csv")

Before OpenAI Request :  0
Before OpenAI Request :  10


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [9]:
import numpy as np
import pandas as pd
import openai
openai.api_key = ""
df = pd.read_csv("embeddings.csv", index_col=0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)
df

,text,embeddings
0,– Blackwell is a graphics processing unit (GP...,"[-0.029476583003997803, -0.01965554617345333, ..."
1,– Named after statistician and mathematician ...,"[-0.014123933389782906, -0.0282897986471653, -..."
6,"– In March 2022, Nvidia announced Hopper arch...","[-0.009168294258415699, -0.041531823575496674,..."
7,– The Blackwell architecture is named after A...,"[-0.012162189930677414, -0.03005015105009079, ..."
8,– In Nvidia's October 2023 Investor Presentat...,"[-0.025766827166080475, -0.03317568078637123, ..."
9,– At the Graphics Technology Conference (GTC)...,"[-0.02770754136145115, -0.024016009643673897, ..."
13,– Blackwell is an architecture designed for b...,"[-0.010315331630408764, -0.0030825245194137096..."
17,– Blackwell is fabricated on the custom 4NP n...,"[-0.0075137377716600895, -0.001662656315602362..."
18,– The GB100 die is at the reticle limit of se...,"[-0.015220609493553638, -0.009573111310601234,..."
25,– CUDA Compute Capability 10.0 is added with ...,"[-0.01434661541134119, -0.03588737174868584, 3..."


In [32]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings
Q1 = "What is successor of Hopper architecture from NVIDIA?"
Q2 = "Explain Blackwell architecture?"
def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine="text-embedding-ada-002")
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [15]:
get_rows_sorted_by_relevance(Q1 , df)

,text,embeddings,distances
8,– In Nvidia's October 2023 Investor Presentat...,"[-0.025766827166080475, -0.03317568078637123, ...",0.121339
6,"– In March 2022, Nvidia announced Hopper arch...","[-0.009168294258415699, -0.041531823575496674,...",0.147348
0,– Blackwell is a graphics processing unit (GP...,"[-0.029476583003997803, -0.01965554617345333, ...",0.171040
9,– At the Graphics Technology Conference (GTC)...,"[-0.02770754136145115, -0.024016009643673897, ...",0.178887
29,– The Blackwell architecture introduces fifth...,"[-0.024566957727074623, -0.009508639574050903,...",0.188123
1,– Named after statistician and mathematician ...,"[-0.014123933389782906, -0.0282897986471653, -...",0.194610
17,– Blackwell is fabricated on the custom 4NP n...,"[-0.0075137377716600895, -0.001662656315602362...",0.205014
33,– List of eponyms of Nvidia GPU microarchitec...,"[-0.018162991851568222, -0.00719132786616683, ...",0.211074
18,– The GB100 die is at the reticle limit of se...,"[-0.015220609493553638, -0.009573111310601234,...",0.213233
25,– CUDA Compute Capability 10.0 is added with ...,"[-0.01434661541134119, -0.03588737174868584, 3...",0.230301


In [16]:
get_rows_sorted_by_relevance(Q2, df)

,text,embeddings,distances
25,– CUDA Compute Capability 10.0 is added with ...,"[-0.01434661541134119, -0.03588737174868584, 3...",0.117361
0,– Blackwell is a graphics processing unit (GP...,"[-0.029476583003997803, -0.01965554617345333, ...",0.154284
9,– At the Graphics Technology Conference (GTC)...,"[-0.02770754136145115, -0.024016009643673897, ...",0.155704
13,– Blackwell is an architecture designed for b...,"[-0.010315331630408764, -0.0030825245194137096...",0.156642
29,– The Blackwell architecture introduces fifth...,"[-0.024566957727074623, -0.009508639574050903,...",0.162701
1,– Named after statistician and mathematician ...,"[-0.014123933389782906, -0.0282897986471653, -...",0.169235
8,– In Nvidia's October 2023 Investor Presentat...,"[-0.025766827166080475, -0.03317568078637123, ...",0.178074
18,– The GB100 die is at the reticle limit of se...,"[-0.015220609493553638, -0.009573111310601234,...",0.205701
17,– Blackwell is fabricated on the custom 4NP n...,"[-0.0075137377716600895, -0.001662656315602362...",0.209182
7,– The Blackwell architecture is named after A...,"[-0.012162189930677414, -0.03005015105009079, ...",0.227008


In [17]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        #print("Token count",current_token_count)
        current_token_count += text_token_count
        #print("Token count",current_token_count)
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)
    

In [20]:
print(create_prompt(Q1, df, 300))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

 – In Nvidia's October 2023 Investor Presentation, its datacenter roadmap was updated to include reference to its B100 and B40 accelerators and the Blackwell architecture. Previously, the successor to Hopper was simply named on roadmaps as "Hopper-Next". Nvidia's updated roadmap emphasized the move from a two-year release cadence for datacenter products to yearly releases targeted for x86 and ARM systems.

###

 – In March 2022, Nvidia announced Hopper architecture for datacenter for AI accelerators. Demand for Hopper products was high throughout 2023's AI hype. The lead time from order to delivery of H100-based servers was between 36 and 52 weeks due to shortages and high demand. Nvidia reportedly sold 500,000 Hopper-based H100 accelerators in Q3 2023 alone. Nvidia's AI dominance with Hopper products led to the company increasing its market capita

In [19]:
print(create_prompt(Q2, df, 300))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

 – CUDA Compute Capability 10.0 is added with Blackwell.

###

 – Blackwell is a graphics processing unit (GPU) microarchitecture developed by Nvidia as the successor to the Hopper and Ada Lovelace microarchitectures.

###

 – At the Graphics Technology Conference (GTC) on March 18, 2024, Nvidia officially announced the Blackwell architecture with focus placed on its B100 and B200 datacenter accelerators. Nvidia CEO Jensen Huang said that with Blackwell, "we created a processor for the generative AI era" and emphasized the overall Blackwell platform combining Blackwell accelerators with Nvidia's ARM-based Grace CPU. Nvidia touted endorsements of Blackwell from the CEOs of Google, Meta, Microsoft, OpenAI and Oracle. The keynote did not mention gaming.

###

 – Blackwell is an architecture designed for both datacenter compute applications and for gam

In [21]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [22]:
custom_Blackwell_answer = answer_question(Q1, df)
print(custom_Blackwell_answer)

The Blackwell architecture is the successor of Hopper architecture from NVIDIA.


In [25]:
Louvre_prompt = f"""
Question: {Q1}
Answer:
"""
initial_Louvre_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=Louvre_prompt,
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_Louvre_answer)

The successor of Hopper architecture from NVIDIA is the Lovelace architecture. It is expected to be released in 2022 and will feature advancements in AI and graphics processing power. It will also use a new chiplet-based design, allowing for more flexibility and scalability in building powerful computing systems. This architecture is expected to be used in both gaming and data center applications.


### Question 2

In [34]:
custom_Blackwell_answer = answer_question(Q2, df)
print(custom_Blackwell_answer)

Blackwell architecture is a graphics processing unit (GPU) microarchitecture developed by Nvidia for datacenter compute applications, gaming, and workstation applications, designed to achieve power efficiency and performance gains through underlying architectural changes. It is named after African American mathematician David Blackwell and utilizes new features such as fifth generation Tensor Cores and support for FP4 and FP6 data types. It is fabricated on the custom 4NP node from TSMC and utilizes the NV-High Bandwidth Interface (NV-HBI) protocol.


In [35]:
Louvre_prompt = f"""
Question: {Q2}
Answer:
"""
initial_Louvre_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=Louvre_prompt,
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_Louvre_answer)

The Blackwell architecture is a framework for designing software systems that consist of a set of interacting components. It is based on the principle of separation of concerns, which means that each component is responsible for a specific set of functions and can be modified or replaced without affecting the overall system.

The architecture is named after David Blackwell, a renowned mathematician who made significant contributions to the fields of probability theory and game theory. He believed that the design of software systems should follow a structured and modular approach, similar to how mathematicians approach problem-solving.

The Blackwell architecture consists of three main components:

1. The Blackwell Core: This is the central component of the architecture, which acts as the coordination point for all other components. It provides a set of
